In [1]:
import numpy as np
import copy
import random
from collections import Counter
from sympy import *
import math
import time

In [2]:
f = open("round_ciphertext_sbox.bin", 'r')
line = f.readline()

In [3]:
f2 = open("round_ciphertext_affine.bin", 'r')
line2 = f2.readline()

In [4]:
f3 = open("round_ciphertext_firstsbox.bin", 'r')
line3 = f3.readline()

In [5]:
ciphertext_sbox_temp = []
value = 0
for i in range(15):
    sbox_result = []
    for j in range(16):
        for k in range(32):
            result = ["0x"+line[l:l+2] for l in range(value, value + 30, 2)]#24 평문
            sbox_result.append(result)
            value += 30
            
    ciphertext_sbox_temp.append(sbox_result)
    
ciphertext_sbox = []
for i in range(15):
    sbox_result2 = []
    for j in range(512):
        temp2 = []
        for k in range(15):
            temp = int(ciphertext_sbox_temp[i][j][k], 16)
            temp2.append(temp)
        sbox_result2.append(temp2)
        
    ciphertext_sbox.append(sbox_result2)

In [6]:
ciphertext_affine_temp = []
value = 0
for i in range(12):
    sbox_result = []
    for j in range(50):
        temp_list = []
        for k in range(10):
            result = ["0x"+line2[l:l+2] for l in range(value, value + 30, 2)]#24 평문
            temp_list.append(result)
            value += 30
        sbox_result.append(temp_list)
            
    ciphertext_affine_temp.append(sbox_result)
    
ciphertext_affine = [[[[0 for a in range(15)] for b in range(10)] for c in range(50)] for d in range(12)]
for i in range(12):
    for j in range(50):
        for k in range(10):
            for l in range(15):
                ciphertext_affine[i][j][k][l] = int(ciphertext_affine_temp[i][j][k][l], 16)

In [7]:
ciphertext_firstsbox_temp = []
value = 0
for i in range(32):
    result = ["0x"+line3[l:l+2] for l in range(value, value + 30, 2)]#24 평문
    ciphertext_firstsbox_temp.append(result)
    value += 30
    
ciphertext_firstsbox = []
for i in range(32):
    temp2 = []
    for j in range(15):
        temp = int(ciphertext_firstsbox_temp[i][j], 16)
        temp2.append(temp)
    ciphertext_firstsbox.append(temp2)

# Function - S-box

In [8]:
def bit5_to_bit60(input): #input= [a1, a2, a3, a4] 5bit X 12
    output = [0 for len in range(60)]
    
    for i in range(12):
        output[5 * i] = (input[i] >> 4) & 1 
        output[5 * i + 1] = (input[i] >> 3) & 1
        output[5 * i + 2] = (input[i] >> 2) & 1
        output[5 * i + 3] = (input[i] >> 1) & 1
        output[5 * i + 4] = input[i] & 1
    
    return output

In [9]:
def bit5_to_bit60(input): #input= [a1, a2, a3, a4] 5bit X 12
    output = [0 for len in range(60)]
    
    for i in range(12):
        output[5 * i] = (input[i] >> 4) & 1 
        output[5 * i + 1] = (input[i] >> 3) & 1
        output[5 * i + 2] = (input[i] >> 2) & 1
        output[5 * i + 3] = (input[i] >> 1) & 1
        output[5 * i + 4] = input[i] & 1
    
    return output

In [10]:
def bit4_to_bit60(input): #input= [a1, a2, a3, a4, a5] 4bit X 15
    output = [0 for len in range(60)]
    
    for i in range(15):
        output[4 * i] = (input[i] >> 3) & 1 
        output[4 * i + 1] = (input[i] >> 2) & 1
        output[4 * i + 2] = (input[i] >> 1) & 1
        output[4 * i + 3] = input[i] & 1
    
    return output

In [11]:
def bit60_to_bit4(input):
    output = []
    for i in range(15):
        value = ((input[4 * i + 3]) & 0x1) + ((input[4 * i + 2] << 1) & 0x3)+((input[4 * i + 1] << 2) & 0x7)+((input[4 * i] << 3) & 0xf)
        output.append(value)
    
    return output

In [12]:
def bit60_to_bit5(input):
    output = []
    for i in range(12):
            value = (input[5 * i + 4] & 0x1) + ((input[5 * i + 3] << 1) & 0x3) + ((input[5 * i + 2] << 2) & 0x7) + ((input[5 * i + 1] << 3) & 0xf) + ((input[5 * i] << 4) & 0x1f)
            output.append(value)
            
    return output

In [13]:
def bit4_to_bit5(input):
    temp = bit4_to_bit60(input)
    output = bit60_to_bit5(temp)
    
    return output

In [14]:
def Matrix_multiplication(Input, Matrix, num1, num2):#Matrix 곱셈 함수
    new_input = []
    temp_input = []
    temp_output = [0 for len in range(60)]
    Output = []
    
    if num1 == 5:
        temp_input = bit5_to_bit60(Input)
                
    elif num1 == 4:
        temp_input = bit4_to_bit60(Input)
    
    for i in range(60):
        for j in range(60):
            temp_output[i] ^= Matrix[i][j] * temp_input[j]

    # 5bit로 분해하는 작업
    if num2 == 5:
        Output = bit60_to_bit5(temp_output)
    
    #4bit로 분해하는 작업
    elif num2 == 4:
        Output = bit60_to_bit4(temp_output)
    
    return Output

In [15]:
def gf2elim(M):
    
    m, n = M.shape

    i = 0
    j = 0
    
    rank = m #max rank = row

    while i < m and j < n:
        # 열 j의 나머지 원소에서 가장 큰 원소의 index와 value 찾기
        k = np.argmax(M[i:, j]) +i

        # swap rows -> M[[k, i]] = M[[i, k]]
        temp = np.copy(M[k])
        M[k] = M[i]
        M[i] = temp

        aijn = M[i, j:]
        col = np.copy(M[:, j]) #M 영향 안받게 복사하는 과정
        col[i] = 0

        flip = np.outer(col, aijn)
        M[:, j:] = M[:, j:] ^ flip

        i += 1
        j += 1
    
    free_val_index = []
    #rank check!
    for i in range(m):
        check = 0
        for j in range(n):
            if M[i, j] == 0:
                check += 1
        if check == n:
            free_val_index.append(i)
            rank -= 1

    return M, free_val_index

In [16]:
def Sbox_attack(input, sbox_num):#input : cipher_text[16 * 32][15], #sbox_num: 공격 sbox index
        
    #1-1. 저장할 Matrix 생성 [16][16] -> sbox 하나에 대한 matrix
    Matrix = [[0 for col in range(16)] for row in range(16)]
        
    #1-2. 나올 수 있는 경우의 수에 해당하여 Matrix 생성 (홀수개)
    for i in range(16):
        for j in range(32):
            Matrix[i][input[32 * i + j][sbox_num]] ^= 1
            
    #1-3. 가우스 소거법 
    Matrix = np.array(Matrix)
    Matrix, free_val_index = gf2elim(Matrix)
    Matrix = Matrix.tolist()
    
    return Matrix, free_val_index

In [17]:
def Rank_Exhaustive_Search_test(Matrix, free_val_index):
    #가우시안 소거법이 적용된 Matrix와 자유 변수 위치
    
    #0. 자유변수를 제외한 나머지 변수의 index를 결정
    remain_val_index = list(range(16))
    for i in free_val_index:
        remain_val_index.remove(i)
    
    free_val_list = list(range(16))
    
    while(True):
        solution = [1 for len in range(16)]
        inv_sbox_list = [0 for len in range(16)]
                
        #1. 자유변수에 랜덤한 값을 넣어주기 위한 리스트 선언
        random.shuffle(free_val_list)
    
        #2. 자유변수 위치에 값을 넣어주는 과정
        for i in range(len(free_val_index)):
            solution[free_val_index[i]] = free_val_list[i]
            inv_sbox_list[free_val_index[i]] = free_val_list[i]
                
        #3. 나머지 값을 결정지어주는 과정
        for i in range(16 - len(free_val_index)):
            for j in range(len(free_val_index)):
                inv_sbox_list[remain_val_index[i]] ^= Matrix[remain_val_index[i]][free_val_index[j]] * solution[free_val_index[j]]
            
        #4. 중복된 원소를 확인해주는 과정
        temp_value = set(inv_sbox_list)
        temp_value = list(temp_value)
        
        if len(temp_value) == 16:
            break
            
    #temp_output = sbox_check(Matrix, inv_sbox_list)
    
    return inv_sbox_list

In [18]:
def main_SAS_sbox_test(ciphertext):
    
    sbox_inv = []
    #ciphertext[15][16 * 32][24]
    for sbox_num in range(15):
        #1. sbox_recover phase
        Matrix, free_val = Sbox_attack(ciphertext[sbox_num], sbox_num)
        print(16 - len(free_val))
        sbox_inv_temp = Rank_Exhaustive_Search_test(Matrix, free_val)
        sbox_inv.append(sbox_inv_temp)


    return sbox_inv    

In [19]:
start = time.time()
Second_sbox_inv = main_SAS_sbox_test(ciphertext_sbox)
end = time.time()
print(f"{end - start:.5f} sec")

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
0.01000 sec


In [20]:
print(Second_sbox_inv)

[[12, 4, 14, 2, 15, 7, 10, 1, 13, 9, 5, 6, 11, 0, 8, 3], [8, 7, 6, 3, 11, 0, 10, 13, 4, 15, 5, 2, 14, 1, 9, 12], [0, 14, 8, 5, 15, 2, 7, 11, 4, 3, 9, 13, 6, 1, 12, 10], [14, 8, 5, 15, 11, 3, 12, 7, 6, 0, 4, 10, 1, 2, 9, 13], [4, 0, 7, 6, 15, 9, 2, 14, 3, 1, 13, 12, 5, 11, 8, 10], [4, 1, 12, 5, 3, 15, 10, 14, 2, 11, 7, 9, 0, 13, 6, 8], [7, 1, 10, 5, 13, 8, 12, 4, 3, 15, 2, 9, 0, 14, 11, 6], [11, 5, 8, 4, 14, 9, 0, 13, 10, 3, 7, 12, 2, 6, 15, 1], [10, 7, 9, 13, 0, 12, 15, 14, 6, 11, 2, 8, 4, 1, 5, 3], [0, 8, 6, 9, 4, 15, 7, 10, 11, 12, 2, 1, 13, 3, 14, 5], [4, 13, 0, 7, 5, 1, 12, 6, 11, 3, 10, 2, 15, 9, 8, 14], [10, 5, 15, 6, 3, 8, 1, 11, 14, 12, 13, 2, 4, 7, 9, 0], [0, 2, 10, 14, 12, 4, 7, 11, 9, 8, 3, 15, 6, 5, 1, 13], [14, 10, 4, 7, 2, 0, 3, 13, 11, 5, 15, 8, 9, 6, 1, 12], [14, 6, 8, 12, 7, 11, 1, 5, 9, 15, 13, 10, 3, 0, 2, 4]]


In [21]:
def find_sbox_inverse(sbox, size):
    result_sbox = [0 for col in range(size)]
    for i in range(size):
        result_sbox[sbox[i]] = i
        
    return result_sbox

In [22]:
Second_sbox = []
for i in range(15):
    Second_sbox.append(find_sbox_inverse(Second_sbox_inv[i], 16))

print("[Second_sbox]")
print(Second_sbox)

[Second_sbox]
[[13, 7, 3, 15, 1, 10, 11, 5, 14, 9, 6, 12, 0, 8, 2, 4], [5, 13, 11, 3, 8, 10, 2, 1, 0, 14, 6, 4, 15, 7, 12, 9], [0, 13, 5, 9, 8, 3, 12, 6, 2, 10, 15, 7, 14, 11, 1, 4], [9, 12, 13, 5, 10, 2, 8, 7, 1, 14, 11, 4, 6, 15, 0, 3], [1, 9, 6, 8, 0, 12, 3, 2, 14, 5, 15, 13, 11, 10, 7, 4], [12, 1, 8, 4, 0, 3, 14, 10, 15, 11, 6, 9, 2, 13, 7, 5], [12, 1, 10, 8, 7, 3, 15, 0, 5, 11, 2, 14, 6, 4, 13, 9], [6, 15, 12, 9, 3, 1, 13, 10, 2, 5, 8, 0, 11, 7, 4, 14], [4, 13, 10, 15, 12, 14, 8, 1, 11, 2, 0, 9, 5, 3, 7, 6], [0, 11, 10, 13, 4, 15, 2, 6, 1, 3, 7, 8, 9, 12, 14, 5], [2, 5, 11, 9, 0, 4, 7, 3, 14, 13, 10, 8, 6, 1, 15, 12], [15, 6, 11, 4, 12, 1, 3, 13, 5, 14, 0, 7, 9, 10, 8, 2], [0, 14, 1, 10, 5, 13, 12, 6, 9, 8, 2, 7, 4, 15, 3, 11], [5, 14, 4, 6, 2, 9, 13, 3, 11, 12, 1, 8, 15, 7, 0, 10], [13, 6, 14, 12, 15, 7, 1, 4, 2, 8, 11, 5, 3, 10, 0, 9]]


# Function - Affine

In [23]:
def basis(matrix):#matrix
    basis_list = []
    temp_list = []
    
    for i in range(len(matrix)):#matrix의 행 크기
        count = 0
        for j in range(5):
            if matrix[i][j] == 0:
                count += 1
        if count != 5:
            temp_list.append(matrix[i])
            
    for i in range(len(temp_list)): #temp_list의 행 크기
        output = []
        for j in range(12):
            value = (temp_list[i][5 * j + 4] & 0x1) + ((temp_list[5 * j + 3] << 1) & 0x3) + ((temp_list[5 * j + 2] << 2) & 0x7) + ((temp_list[5 * j + 1] << 3) & 0xf) + ((temp_list[5 * j] << 4) & 0x1f)
            output.append(value)
        basis_list.append(output)
        
    return basis_list

In [24]:
def Basis_check(input):#input의 형태는 Affine의 출력값이 들어가게 된다. [a1, a2, a3, a4, a5], ..., [] -> 5개
    
    my_list = []
    input_60bit = []
    
    #input의 형태를 20bit X 20bit로 변형해주는 함수
    for i in range(len(input)):
        input_temp = bit4_to_bit60(input[i])
        input_60bit.append(input_temp)
        
    print("[input_matrix]")
    print(input)
    
    print("[input_matrix_60bit]")
    print(input_60bit)
    print("\n")
    
    #Guassian elimination 
    input_60bit = np.array(input_60bit)
    M, free_val = gf2elim(input_60bit)
    M = M.tolist()
    
    rank_len = 5 - len(free_val)
    
    for i in range(5):
        for j in range(i + 1, 5):
            if M[i] == M[j]:
                rank_len -= 1
                for k in range(60):
                    M[i][k] = 0
    
    print("[glim_matrix]")
    print(M)
    
    return rank_len, M

In [25]:
def invert(Matrix):#Matrix의 크기는 Matrix[40][40]
    result = copy.deepcopy(Matrix)
    
    for k in range(60):#순수 matrix size
        if result[k][k] == 0:
            for i in range(k, 60):
                if result[i][k] != 0:
                    for x in range(120):
                        t = result[k][x]
                        result[k][x] = result[i][x]
                        result[i][x] = t
                        
            if result[k][k] == 0:
                return 0
            
        for i in range(60):
            if i != k:
                if result[i][k]:
                    for j in range(k, 120):
                        result[i][j] ^= result[k][j]
        
    return result

In [26]:
def movMat(A_matrix):
    B_matrix = [[0 for col in range(120)] for row in range(120)]
    
    for i in range(120):
        for j in range(120):
            B_matrix[i][j] = A_matrix[i][j]
    
    return B_matrix

In [27]:
def molMat(A_matrix, B_matrix):#A_matrix, B_matrix[40][40]
    result = [[0 for col in range(120)] for row in range(120)]
    
    for i in range(60):
        for j in range(60):
            result[i][j] = 0
            for x in range(60):
                result[i][j] ^= (A_matrix[i][x] * B_matrix[x][j])
    
    return result

In [28]:
def Identity_matrix_check(matrix):
    
    return_value = 0
    for i in range(60):#matrix size
        for j in range(60):#matrix size
            if i == j:
                if matrix[i][j] != 1:
                    return_value = 1
            else:
                if matrix[i][j] != 0:
                    return_value = 1
                    
    return return_value

In [29]:
def inverse_matrix_check(Matrix):#Matrix[20][20]
    
    matrix_temp = [[0 for col in range(120)] for row in range(120)]
    
    for i in range(60):
        for j in range(60):
            matrix_temp[i][j] = Matrix[i][j]
    
    for i in range(60):
        for j in range(120):
            if j == (i + 60):
                matrix_temp[i][j] = 1
    
    inverse_result = invert(matrix_temp)
    if inverse_result == 0:
        print("fail!")
        return 1, "fail!"
    else:
        for i in range(60):
            for j in range(60):
                inverse_result[i][j] = inverse_result[i][j + 60]
            
        mul_result = molMat(matrix_temp, inverse_result)
    
        if Identity_matrix_check(mul_result) == 0:
            print("[Inverse_matrix find success!]")
            result = []
            for i in range(60):
                result.append(copy.deepcopy(inverse_result[i][:60]))
            return result, "[Inverse_matrix find success!]"
        else:
            print("[Inverse_matrix find fail!]")
            return 1, "[[Inverse_matrix find fail!]]"

In [30]:
def basis_range(ge_affine):
    range_result = []
    for i in range(60):
        for j in range(len(ge_affine)):
            if ge_affine[j][i] == 1:
                if ge_affine[j] not in range_result:
                    range_result.append(ge_affine[j])
    
    return range_result

In [31]:
def main_SAS_affine(Second_sbox_inv, ciphertext):
    result = ""
    count_list = [0 for len in range(12)]   
    while result != "[Inverse_matrix find success!]":
        find_affine = [[0 for col in range(60)] for row in range(60)]
        for m in range(12):
            rank = 0
            while(rank != 5):
                temp_list0 = []
                for i in range(50):
                    temp_list1 = []
                    for j in range(10):
                        temp_list2 = []
                        for l in range(15):
                            temp_list2.append(Second_sbox_inv[l][ciphertext[m][i][j][l]])
                        temp_list1.append(temp_list2)
                    temp_list0.append(temp_list1)
                            
                XOR_ciphertext = [[0 for col in range(15)] for row in range(5)]
                for i in range(5):
                    for j in range(15):
                        XOR_ciphertext[i][j] = temp_list0[count_list[m]][i * 2][j] ^ temp_list0[count_list[m]][i * 2 + 1][j]
                rank, ge_affine = Basis_check(XOR_ciphertext)
                
                ge_affine = basis_range(ge_affine)
                count_list[m] += 1
            for k in range(60):
                for i in range(5):
                    find_affine[k][5 * m + i] = ge_affine[i][k]
            
        find_affine_inv, result = inverse_matrix_check(find_affine)
        
    print(find_affine)
    
    return find_affine, find_affine_inv

find_affine = main_SAS_affine(Second_sbox_inv, ciphertext_affine)

recovery_ciphertext = []

for i in range(12):
    sbox_result = []
    for j in range(500):
        temp = []
        for k in range(10):
            temp.append(Second_sbox_inv[k][ciphertext_affine[i][j][k][l]])
            
        sbox_result.append(temp)
        
    recovery_ciphertext.append(sbox_result)

In [32]:
start = time.time()
find_affine, find_affine_inv = main_SAS_affine(Second_sbox_inv, ciphertext_affine)
end = time.time()
print(f"{end - start:.5f} sec")

[input_matrix]
[[13, 11, 13, 12, 12, 12, 10, 9, 11, 12, 15, 9, 6, 3, 6], [0, 8, 3, 9, 0, 11, 15, 9, 14, 2, 4, 7, 0, 10, 10], [2, 13, 10, 4, 6, 0, 0, 3, 11, 15, 12, 9, 14, 10, 12], [2, 5, 9, 13, 6, 11, 15, 10, 5, 13, 8, 14, 14, 0, 6], [12, 11, 1, 15, 0, 10, 4, 3, 0, 4, 8, 7, 10, 5, 7]]
[input_matrix_60bit]
[[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1

start = time.time()
find_affine = main_SAS_affine(Second_sbox_inv, recovery_ciphertext)
end = time.time()
print(f"{end - start:.5f} sec")

In [33]:
find_affine_inv, result = inverse_matrix_check(find_affine)

[Inverse_matrix find success!]


# Function - first sbox

In [34]:
def First_sbox_recovery(Second_sbox_inv, find_affine_inv, ciphertext):#ciphertext[32][15]
    test_plaintext = [0 for len in range(12)]
    First_sbox = [[], [], [], [], [], [], [], [], [], [], [], []]
    for i in range(32):
        print(ciphertext[i])
        temp_sbox2 = []
        for j in range(15):
            temp_sbox2.append(Second_sbox_inv[j][ciphertext[i][j]])
        
        print(temp_sbox2)
        temp_sbox2_60bit = bit4_to_bit60(temp_sbox2)
    
        temp_affine = [0 for len in range(60)]
        for j in range(60):
            for k in range(60):
                temp_affine[j] ^= find_affine_inv[j][k] * temp_sbox2_60bit[k]
                
    
        temp_affine_5bit = bit60_to_bit5(temp_affine)
        print(temp_affine_5bit)
        print("\n")
        for j in range(12):
            First_sbox[j].append(temp_affine_5bit[j])
    
    return First_sbox


In [35]:
First_sbox = First_sbox_recovery(Second_sbox_inv, find_affine_inv, ciphertext_firstsbox)

[4, 6, 5, 4, 10, 4, 14, 2, 11, 9, 1, 9, 12, 1, 14]
[15, 10, 2, 11, 13, 3, 11, 8, 8, 12, 13, 12, 6, 10, 2]
[27, 2, 13, 3, 0, 28, 6, 16, 20, 10, 3, 27]


[1, 9, 5, 8, 2, 6, 1, 12, 0, 2, 4, 2, 2, 10, 12]
[4, 15, 2, 6, 7, 10, 1, 2, 10, 6, 5, 15, 10, 15, 3]
[11, 7, 30, 17, 11, 24, 13, 15, 12, 5, 14, 16]


[2, 7, 10, 5, 10, 3, 12, 4, 13, 13, 6, 6, 10, 3, 5]
[14, 13, 9, 3, 13, 5, 0, 14, 1, 3, 12, 1, 3, 7, 11]
[14, 4, 4, 21, 23, 14, 1, 24, 14, 20, 7, 22]


[10, 4, 5, 6, 15, 6, 10, 7, 1, 14, 12, 5, 0, 14, 14]
[5, 11, 2, 12, 10, 10, 2, 13, 7, 14, 15, 8, 0, 1, 2]
[3, 9, 2, 23, 12, 19, 26, 27, 11, 11, 19, 29]


[4, 9, 8, 12, 1, 9, 3, 0, 6, 10, 15, 0, 15, 5, 5]
[15, 15, 4, 1, 0, 11, 5, 11, 15, 2, 14, 10, 13, 0, 11]
[2, 23, 18, 10, 21, 2, 23, 8, 16, 1, 4, 9]


[1, 3, 9, 3, 5, 15, 9, 1, 3, 0, 2, 1, 4, 9, 15]
[4, 3, 3, 15, 9, 8, 15, 5, 13, 0, 0, 5, 12, 5, 4]
[30, 14, 8, 2, 8, 7, 29, 17, 19, 15, 25, 20]


[12, 6, 1, 2, 14, 2, 0, 12, 0, 14, 12, 0, 13, 10, 4]
[11, 10, 14, 5, 8, 12, 7, 2, 10, 14, 15, 10, 

In [36]:
print(First_sbox)
for i in range(12):
    my_set = set(First_sbox[i])
    my_list = list(my_set)

[[27, 11, 14, 3, 2, 30, 18, 4, 12, 8, 20, 10, 15, 24, 26, 28, 25, 1, 17, 5, 9, 29, 21, 7, 22, 6, 19, 16, 31, 0, 13, 23], [2, 7, 4, 9, 23, 14, 17, 22, 26, 11, 27, 0, 3, 1, 15, 20, 18, 19, 24, 5, 16, 28, 10, 12, 8, 29, 21, 13, 6, 25, 31, 30], [13, 30, 4, 2, 18, 8, 26, 12, 6, 9, 31, 0, 17, 22, 3, 28, 15, 21, 14, 25, 7, 1, 29, 10, 20, 23, 5, 16, 11, 24, 27, 19], [3, 17, 21, 23, 10, 2, 22, 5, 27, 8, 13, 15, 4, 11, 25, 30, 1, 28, 19, 26, 0, 24, 29, 7, 12, 18, 31, 14, 20, 9, 16, 6], [0, 11, 23, 12, 21, 8, 5, 22, 27, 7, 26, 20, 15, 24, 6, 18, 16, 9, 1, 3, 30, 31, 17, 19, 13, 14, 28, 4, 25, 10, 29, 2], [28, 24, 14, 19, 2, 7, 5, 29, 4, 12, 31, 13, 22, 1, 6, 26, 20, 11, 23, 9, 16, 25, 3, 0, 21, 15, 10, 27, 18, 17, 8, 30], [6, 13, 1, 26, 23, 29, 10, 4, 9, 28, 21, 3, 16, 8, 17, 20, 19, 18, 11, 27, 12, 30, 0, 5, 2, 15, 14, 24, 7, 25, 31, 22], [16, 15, 24, 27, 8, 17, 13, 0, 4, 29, 10, 11, 20, 31, 14, 3, 30, 12, 6, 22, 1, 2, 21, 28, 18, 7, 5, 23, 26, 9, 25, 19], [20, 12, 14, 11, 16, 19, 17, 7, 21, 13,

In [37]:
First_sbox_inv = []
for i in range(12):
    First_sbox_inv.append(find_sbox_inverse(First_sbox[i], 32))

print("[First_sbox_inv]")
print(First_sbox_inv)

[First_sbox_inv]
[[29, 17, 4, 3, 7, 19, 25, 23, 9, 20, 11, 1, 8, 30, 2, 12, 27, 18, 6, 26, 10, 22, 24, 31, 13, 16, 14, 0, 15, 21, 5, 28], [11, 13, 0, 12, 2, 19, 28, 1, 24, 3, 22, 9, 23, 27, 5, 14, 20, 6, 16, 17, 15, 26, 7, 4, 18, 29, 8, 10, 21, 25, 31, 30], [11, 21, 3, 14, 2, 26, 8, 20, 5, 9, 23, 28, 7, 0, 18, 16, 27, 12, 4, 31, 24, 17, 13, 25, 29, 19, 6, 30, 15, 22, 1, 10], [20, 16, 5, 0, 12, 7, 31, 23, 9, 29, 4, 13, 24, 10, 27, 11, 30, 1, 25, 18, 28, 2, 6, 3, 21, 14, 19, 8, 17, 22, 15, 26], [0, 18, 31, 19, 27, 6, 14, 9, 5, 17, 29, 1, 3, 24, 25, 12, 16, 22, 15, 23, 11, 4, 7, 2, 13, 28, 10, 8, 26, 30, 20, 21], [23, 13, 4, 22, 8, 6, 14, 5, 30, 19, 26, 17, 9, 11, 2, 25, 20, 29, 28, 3, 16, 24, 12, 18, 1, 21, 15, 27, 0, 7, 31, 10], [22, 2, 24, 11, 7, 23, 0, 28, 13, 8, 6, 18, 20, 1, 26, 25, 12, 14, 17, 16, 15, 10, 31, 4, 27, 29, 3, 19, 9, 5, 21, 30], [7, 20, 21, 15, 8, 26, 18, 25, 4, 29, 10, 11, 17, 6, 14, 1, 0, 5, 24, 31, 12, 22, 19, 27, 2, 30, 28, 3, 23, 9, 16, 13], [25, 10, 13, 17, 15, 1

In [38]:
with open('6round_inverse.bin', 'wb') as f:
    for i in range(12):
        for j in range(32):
            f.write(bytes([First_sbox_inv[i][j]]))
            
    for i in range(60):
        for j in range(60):
            f.write(bytes([find_affine_inv[i][j]]))
            
    for i in range(15):
        for j in range(16):
            f.write(bytes([Second_sbox_inv[i][j]]))
            
f.close()